# <center>IOITC 2021 Notes</center>
Meeting ID: 818 8493 8161<br>
Passcode: 800895<br>
Meeting link: [us02web.zoom.us/j/81884938161?pwd=alNuY0o4ZFBUdi9CZmQwdnpwWDMyUT09](https://us02web.zoom.us/j/81884938161?pwd=alNuY0o4ZFBUdi9CZmQwdnpwWDMyUT09)<br>
Recordings: [youtube.com/playlist?list=PLFY28j3a734SJ6V88HG0ArSFarqHIyHCW](https://www.youtube.com/playlist?list=PLFY28j3a734SJ6V88HG0ArSFarqHIyHCW)<br>
Lecture Notes: [cmi.ac.in/~kumar/IOITC2021/](https://www.cmi.ac.in/~kumar/IOITC2021/)

# Prefix Sums and Sliding Windows
## Problem:
You are given an array A with n elements: a\[0\], a\[1\] ... a\[n - 1\] (a\[i\] >= 0). The task is to find the longest segment with a sum <= k.

## Solution:
Use a sliding window to find the longest segment.
Keep two indices l and r. If the sum between them is less than or equal to k increment r else increment l

The total number of steps is not more than 2n so the solution has a O(n) time complexity

In [1]:
a = [2, 1, 3, 1, 4, 4, 1, 8, 1, 7]
n = len(a)
k = 9

l = 0
r = 0

maxLength = 0
currSum = a[0]

while l <= r and r < len(a):
    if currSum > k:
        #Shrink the segment from the left
        currSum -= a[l]
        l += 1
        continue

    maxLength = max(maxLength, r - l + 1)

    #Expand the segment to the right
    r += 1
    if r < len(a):
        currSum += a[r]

print(f'The maximum length segment is {maxLength}')

The maximum length segment is 4


## Problem:
The problem changes if a\[i\] is allowed to be negative
Here there are 2 possible solutions

## Solution:
Create a list of prefix sums and sort them along with their indices.
For every segment starting at index l we have to find the greatest index r such that
<center>prefSum[r + 1] - prefSum[l] <= k<br>
prefSum[r + 1] <= k + prefSum[l]</center><br>
First we can binary search for the prefSum[r]
We can create a new array which is the largest index in the prefix array of the prefix sum array

Sorting the prefix sum array takes O(nlogn) time and we have to binary search for each index which is also O(n) * O(logn) = O(nlogn).<br>
So the final time complexity is O(nlogn)

In [2]:
from bisect import bisect_right as upperBound

a = [2, 1, -3, 1, 4, -4, 1, -8, 1, 7]
n = len(a)
k = 9

prefSum = [(0, -1)] * (n + 1)
for i in range(1, n + 1):
    prefSum[i] = (prefSum[i - 1][0] + a[i - 1], i)
origPrefSum = prefSum.copy()
prefSum.sort()

maxPrefixIndex = [prefSum[0][1]] * (n + 1)
for i in range(1, n):
    maxPrefixIndex[i] = max(maxPrefixIndex[i - 1], prefSum[i + 1][1])

maxLength = 0
for l in range(n):
    maxPrefixSumIndex = upperBound(prefSum, (k + origPrefSum[l][0], n)) - 1
    r = maxPrefixIndex[maxPrefixSumIndex]
    maxLength = max(maxLength, r - l)

print(f'The maximum length segment is {maxLength}')

The maximum length segment is 8


## Problem:
Given an array a: a\[0\], a\[1\] ... a\[n - 1\] (a\[i\] >= 0), find the shortest pair of disjoint segments each adding up to k

## Solution:
Create an array b where b\[i\] is the length of the shortest segment adding to k starting at i.<br>
Then create another array e where e\[i\] is the length of the shortest segment adding to k ending at i.

Create two more arrays b1 and e1 where <br>
b1\[i\] = min(b\[i\], b\[i + 1\] ... b\[n - 1\]) and <br>
e1\[i\] = min(1\[0\], e\[1\] ... e\[i\])

For each i if we take one segment which ends before i and one which starts after i, we can find the answer to the problem. <br>
This is equal to e1\[i - 1\] + b1\[i + 1\]

In [3]:
a = [2, 1, 3, 1, 4, 4, 1, 8, 1, 7]
n = len(a)
k = 4

b = [n] * n
e = [n] * n

l = r = 0
currSum = a[0]
while l <= r and r < n:
    if currSum > k:
        currSum -= a[l]
        l += 1
        continue
    
    if currSum == k:
        b[l] = min(b[l], r - l + 1)
        e[r] = min(e[r], r - l + 1)
    
    r += 1
    if r < n:
        currSum += a[r]

b1 = b.copy()
for i in range(n - 2, -1, -1):
    b1[i] = min(b1[i], b1[i + 1])

e1 = e.copy()
for i in range(1, n):
    e1[i] = min(e1[i], e1[i - 1])

res = 2 * n
for i in range(1, n - 1):
    res = min(res, e1[i - 1] + b1[i + 1])

if res == 2 * n:
    print('There are no possible segments')
else:
    print(f'The total lengths of the segments are {res}')

The total lengths of the segments are 3


## Problem: (Garden IOI 2005)
There is a grid with some roses in it. Each cell in the grid is either filled with a rose or not. Find the smallest perimeter rectangle which has k roses inside it.

## Solution:
Create a prefix sum for the grid. <br>
prefSum\[x\]\[y\] = prefSum\[x - 1\]\[y\] + prefSum\[x\]\[y - 1\] - prefSum\[x - 1\]\[y - 1\] + a\[i - 1\]\[j - 1\]

For each pair of rows l, r (l <= r) we perform a sliding window operation between those 2 rows to see which columns fit. For each rectangle that has exactly k roses we find the perimeter of the rectange and check if it less than the current minimum

The time complexity if O(n^2m) because we iterate through every pair of rows and perform a sliding window operation which takes O(m) time

In [4]:
a = [
    [0, 1, 1],
    [1, 0, 0],
    [0, 0, 1],
]
n = len(a)
m = len(a[0])
k = 2

prefSum = [[0] * (m + 1) for i in range(n + 1)]
for i in range(1, n + 1):
    for j in range(1, m + 1):
        prefSum[i][j] = prefSum[i - 1][j] + prefSum[i][j - 1] - prefSum[i - 1][j - 1] + a[i - 1][j - 1]

res = -1
for y1 in range(n):
    for y2 in range(y1, n):
        x1 = 0
        x2 = 0
        while x1 <= x2 and x2 < m:
            currSum = prefSum[x2 + 1][y2 + 1] - prefSum[x2 + 1][y1] - prefSum[x1][y2 + 1] + prefSum[x1][y1]

            if currSum > k:
                x1 += 1
                continue

            if currSum == k:
                perimeter = (y2 - y1 + 1) * 2 + (x2 - x1 + 1) * 2
                if res == -1 or res > perimeter:
                    res = perimeter
            
            x2 += 1
if res == -1:
    print('There is no rectangle with k roses')
else:
    print(f'The minimum valid perimeter is {res}')

The minimum valid perimeter is 6


## Problem:
You are given an array a with n integers.
Answer > n queries of the form what is the minimum value in the range a\[l\], a\[l + 1\] ... a\[r\]

## Solution:
Precalculate the minimum of every segment that has a length which is an integral length of 2 (1, 2, 4 ...)
Let b\[i\]\[j\] be the minimum of the segment with length 2 ^ j starting at i

<center>b[i][0] = a[i]<br>
b[i][j] = min(b[i][j - 1], b[i + 2 ^ (j - 1)][j - 1]) (for j > 0)</center>

Given a query (l, r) we must spilt l, r into segments that are powers of 2. Find the largest k such that i + 2 ^ k <= j. The answer is min(b\[i\]\[k\], b\[j - 2 ^ k\]\[k\])

In [5]:
from math import ceil, log

a = [2, 1, 3, 3, 4, 4, 9, 8, 1, 7]
n = len(a)

queries = [(2, 5)]
q = len(queries)

h = int(ceil(log(n, 2)))
b = [[0] * (h + 1) for i in range(n)]
for i in range(n):
    b[i][0] = a[i]
curr = 1
for l in range(1, h + 1):
    for i in range(n - curr):
        b[i][l] = min(b[i][l - 1], b[i + curr][l - 1])
    curr *= 2

for i in range(q):
    l, r = queries[i]
    k = int(log(r - l + 1, 2))
    res = min(b[l][k], b[r - 2 ^ k + 1][k])
    print(f'The minimum element in the range is {res}')

The minimum element in the range is 1


## Problem:
Given an array of length n and an integer k, answer q queries of the type: What is the minimum element starting at index i and with a length k.

## Solution:
Split the array into subarrays of length k with a smaller segment at the end. For each subarray, calculate the minimum value from the begining of the current segment and the minimum value from the end of the current segment. For an index i, the answer will be:
<center>min(r[i], l[i + k - 1])</center>

In [6]:
k = 2
a = [2, 1, 3, 3, 4, 4, 9, 8, 1, 7]
n = len(a)

l = [0] * n
r = [0] * n

for i in range(0, n, k):
    l[i] = a[i]
    for j in range(i + 1, min(n, i + k)):
        l[j] = min(l[j - 1], a[j])
    
    r[min(n, i + k) - 1] = a[min(n, i + k) - 1]
    for j in reversed(range(i, min(n, i + k) - 1)):
        r[j] = min(r[j + 1], a[j])

queries = [2, 5, 3, 6]

for q in queries:
    print(min(r[q], l[q + k - 1]))

3
4
3
8


## Problem: (IOI 2006 Pyramids)
You are given a grid of size n \* m. Each cell in the grid has a height. You have to choose an outer rectangle of dimensions a \* b and an inner rectangle of dimensions c \* d.

Choose two rectangles such that the average of cells in the a \* b rectangle but not in the c \* d rectangle is maximized.

## Solution:
Since every pair of rectangles has (a \* b) - (c \* d) cells to average, we an maximize the sum instead of the average.

# Segment Tree
Segment Trees support 2 operations:
1. Update the value at index to x
2. Give the result for a function over a range \[i, j\] (function should be commutative e.g. +, min, max etc.)

## Structure of the Segment Tree
The segment tree is in the structure of a complete binary tree. This tree has a height of log(n) where n is the length of the array. Every leaf represents a single element in the array. For every other node, it contains the answer for a range \[l, r\] where (r - l + 1) is an integral power of 2. If the middle of this range is m = (l + r) / 2, the left child contains the answer for \[l, m\] and the right child contains the answer for \[m + 1, r\]. This binary tree can be stored as an array where the children of the node at i are 2 \* i and 2 \* i + 1. The parent of the node at i is floor(i / 2)

## Update the value at a given index
We go to the node in the tree which corresponds to the given index and update the value at that position. The we go to the parent of this node and update the answer for this node. Go to its parent and repeat until we reach the root of the tree. Since the height of the tree is log(n), the operation takes log(n) time.

## Range query
We are given a range \[l, r\] and we are asked to find the answer for this range. We start at the root and recursively move to the left and right children. At any point is the current range is outside the given range we return an indentity value. Otherwise, if the current range is entirely inside the given range, we return the value at this node. We combine the value one both halves and return the value.

In [7]:
class SegTree:
    segTree = []
    
    def __init__(self, a):
        self.segTree = [0] * (4 * len(a))
        for i in range(len(a)):
            self.update(i, a[i], 1, 0, len(a) - 1)
    
    def update(self, i, x, curr, l, r):
        if l == i and i == r:
            self.segTree[curr] = x
            return
        
        m = (l + r) // 2
        if i <= m:
            self.update(i, x, curr * 2, l, m)
        else:
            self.update(i, x, curr * 2 + 1, m + 1, r)
        
        self.segTree[curr] = self.segTree[2 * curr] + self.segTree[2 * curr + 1]
    
    def query(self, l, r, curr, s, e):
        if r < s or e < l:
            return 0
        
        if l <= s and e <= r:
            return self.segTree[curr]
        
        m = (s + e) // 2
        return self.query(l, r, curr * 2, s, m) + self.query(l, r, curr * 2 + 1, m + 1, e)

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
n = len(a)

segTree = SegTree(a)

print(segTree.query(4, 9, 1, 0, n - 1))
segTree.update(7, 15, 1, 0, n - 1)
print(segTree.query(4, 9, 1, 0, n - 1))
print(segTree.query(2, 3, 1, 0, n - 1))

45
52
7


# Binary Index Tree

## Structure of the Binary Index Tree
For each index i (0 <= i <= n), let k be the largest integer such that i % 2 ^ k == 0. At index i in the binary index tree, we store the sum for the segment of length 2 ^ k ending at i (Here we use 1 based indexing). For example for i == 3 we store the value at index 3 and for index 6 we store the sum of the value at index and the value at index 6.

## Increase the element at a given index by a given value
For this operation we would have have to update the value at i and some indices j (j > i). Let k be the largest integer such that i % 2 ^ k == 0. The next index j that we have to update is the first index greater than i for which there exists and integer k' such that
<center>j % 2 ^ k' == 0 and j % 2 ^ (k' + 1) != 0 and k' >= k</center><br>
We make i = j and repeat the step until we reach and invalid i. For each index we add the value in the binary tree by the given value.

## Increasing the value at a given index by a particular value
We can use this information to calculate the prefix sum for any index i. For each value i, we calculate an index the largest index which is outside the segment stored in the binary index tree at index i.
<center>j = i - 2 ^ k (where k = the largest integer such that i % 2 ^ k == 0)</center><br>
This can also be perfomed by flipping the last 1 from the right into a 0. We add the value in the binary index tree at value i to our prefix and repeat the step with j = i until we reach i = 0. The sum of all the values is the prefix sum. Since the index can have at most logn ones in its binary represeentation and thereforre the total number of jumps is no more than logn, the time complexity of this operation is O(logn).

In [8]:
def getsum(BITTree,i):
    s = 0
    i = i+1

    while i > 0:
        s += BITTree[i]
        i -= i & (-i)

    return s

def updatebit(BITTree , n , i ,v):
    i += 1
    while i <= n:
        BITTree[i] += v
        i += i & (-i)

def construct(arr, n):

    BITTree = [0]*(n+1)

    for i in range(n):
        updatebit(BITTree, n, i, arr[i])

    return BITTree

freq = [2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9]

BITTree = construct(freq,len(freq))
print(f'Sum of elements in arr[0..5] is {getsum(BITTree,5)}')

freq[3] += 6
updatebit(BITTree, len(freq), 3, 6)

print(f'Sum of elements in arr[0..5] after update is {getsum(BITTree,5)}')

Sum of elements in arr[0..5] is 12
Sum of elements in arr[0..5] after update is 18


# Difference Array

## Structure of the Difference Array
Given an array a with length n, d\[0\] == a\[0\] and for each index i (0 < i < n) d\[i\] = a\[i\] - a\[i - 1\].

## Increase all the values in a given range by a given value
If we have to increase all elements in the range \[l, r\] by x we can just increase d\[r\] by x and decrease d\[l - 1\] by x.

## Recover the array after the updates
We know that a\[0\] = d\[0\]. For all i (0 < i < n) a\[i\] = d\[i\] + a\[i - 1\].

In [9]:
class DifferenceArray:
    diffArr = []

    def __init__(self, a):
        n = len(a)
        self.diffArr = [a[0]] * n + [0]
        for i in range(1, n):
            self.diffArr[i] = a[i] - a[i - 1]

    def update(self, l, r, x):
        self.diffArr[r + 1] -= x
        self.diffArr[l] += x
    
    def getArr(self):
        a = [self.diffArr[0]] * (len(self.diffArr) - 1)
        for i in range(1, n):
            a[i] = self.diffArr[i] + a[i - 1]
        return a

a = [2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9]

differenceArray = DifferenceArray(a)
print(*a)

differenceArray.update(0, 5, 2)
a = differenceArray.getArr()
print(*a)

2 1 1 3 2 3 4 5 6 7 8 9
4 3 3 5 4 5 4 5 6 7 4 4


# Graphs

## Spanning Tree
A spanning tree is a subset of edges of from a graph such that the number a tree is formed and all nodes are connected

## Minimum Spanning Tree
The minimum spanning tree is a spanning tree of the graph which has the least sum of weights in its edges. To find the minimum spanning tree, we sort all edges in ascending order of weight. For each edge if we can use it to connect two unconnected components, we add it to our minimum spanning tree. We keep adding edges until we have formed out tree.

In [10]:
class DSU:
    e = []

    def __init__(self, n):
        self.e = [-1] * n
    
    def get(self, x):
        if self.e[x] < 0:
            return x
        self.e[x] = self.get(self.e[x])
        return self.e[x]
    
    def unite(self, x, y):
        x = self.get(x)
        y = self.get(y)

        if x == y:
            return False
        
        if self.e[x] > self.e[y]:
            x, y = y, x
        
        self.e[x] += self.e[y]
        self.e[y] = x
        return True

n = 5
m = 8

edges = [
    (0, 1, 2),
    (1, 2, 1),
    (2, 0, 3),
    (2, 4, 9),
    (1, 3, 8),
    (2, 3, 1),
    (0, 4, 6),
    (1, 3, 5)
]

edges.sort(key = lambda x: x[2])
dsu = DSU(n)
mst = []

for u, v, w in edges:
    if dsu.unite(u, v):
        mst.append((u, v, w))
        if len(mst) == n - 1:
            break

res = [[] for _ in range(n)]
for u, v, w in mst:
    res[u].append((v, w))
    res[v].append((u, w))

for u in range(n):
    print(u, end = ': ')
    for v, w in res[u]:
        print(f'({v}, {w})', end = ' ')
    print()

0: (1, 2) (4, 6) 
1: (2, 1) (0, 2) 
2: (1, 1) (3, 1) 
3: (2, 1) 
4: (0, 6) 


# Game Theory
Combinatorial Game Theory: When there is perfect and no chance is involved.
## Conditions of a game
 - Game must always end
 - Two players are involved
 - Fixed starting conditions
 - Only win/lose endings (no draw)

Example game:
 - There are N stones in a pile
 - In each turn, the player can choose to remove 1 / 2 stones
 - The person to remove the last stone wins

## Game states
Each position of the game can be defined as a state. Each state can be classified as a winning state or a losing state.

Terminal State: End state for which we know the verdict (winning / losing)<br>
Winning State: State from which we can move to a losing state<br>
Losing State: State from which we cannot move to a losing state<br>

## Problem:
We are given a tree which a value written on it. We start at the root and in each turn we move down to one of its children. There are 2 players, the first player wants to minimize the final number and the second player wants to maximize the final number. Find the final number if both players play optimaly.
## Solution
We start at the root and calculate the distance to every node using dfs. We also create a dp array. For every leaf node the dp value is the value on the leaf node. For every other node if the distance to that node is divisible by 2 is 0, the dp value is the minimum dp value of all its children. Otherwise the dp value is the maximum of all its children. The final answer is the dp value of the root.

## Problem: (Nim's game)
We have n piles of coins each with a\[i\] coins each. In every turn the player can choose a pile and remove any number of coins from that pile. The person who removes the last coin wins.

## Solution:
If the XOR sum of the array a is 0 it is a losing state, else it is a winning state.

## Proof:
We know that \[0, 0, 0 ...\] is a losing state.<br>
For losing state:
<center>
We know a[0] ^ a[1] ^ ... ^ a[n - 1] = 0<br>
Then, a[0] = a[1] ^ a[2] ^ ... ^ a[n - 1]<br>
If we remove x (x > 0) from a[0], we get a[0] - x<br>
Then we get (a[0] - x) ^ a[1] ^ a[2] ^ ... ^ a[n - 1]<br>
Which is equal to (a[0] - x) ^ a[0]<br>
Since a[0] - x != a[0], (a[0] - x) ^ a[0] != 0 which is a winning state
</center>

For winning state:
<center>
We know a[0] ^ a[1] ^ ... ^ a[n - 1] != 0<br>
If we remove x stones from a[0] we get a[0] - x<br>
The final XOR sum is (a[0] - x) ^ a[1] ^ ... ^ a[n - 1]<br>
We have to find a value of x such that (a[0] - x) ^ a[1] ^ ... ^ a[n - 1] = 0<br>
or x = a[0] - (a[1] ^ a[2] ^ ... ^ a[n - 1])<br>
Since a[1] ^ a[2] ^ a[n - 1] is non-negative we know that x <= a[0] and thus is a valid number of stones to remove
</center>

## Sprague-Grundy Theorem:
Playing and impartial game is the equivalent of playing the nim game with the grundy values of the game.
The grundy value of a losing state is 0 and the grundy value of a winning state is positive. For every state, the grundy value of that state is the MEX of the grundy values that are reachable in one moves.